# Final Project: RUSH Business Case Analysis
Class: GEN BUS 885\
Author: Matthew Brown\
Date: 07/29/2025

## Business Scenario
You work as a sales analyst for RUSH, a globally renowned sportswear and footwear brand known for its innovative designs and performance-oriented products. The company stores its raw sales data as a collection of three tables:
+ TABLE_PRODUCTS
+ TABLE_RETAILER
+ TABLE_SALES

The data includes the number of units sold, the total sales revenue, the location of the sales, the type of product sold, as well as other relevant information. (For data field definitions and explanations, see the data dictionary.) The data is "raw," meaning it has not been cleaned and probably contains errors that need to be addressed.

The VP of US Sales has tasked you with analyzing sales data for trends and insights that will help company leadership understand the market and identify opportunities for growth. For example, you may want to look for trends or insights in seasonality, retailers, locations, or sales methods. Take initiative to apply your creativity and curiosity to this data.

In addition, she has asked you to answer the following business questions:
1. What product category (product) had the highest sales (in dollars) in 2021? How much did it sell?
2. What state had the highest sales (in dollars) of women's products in 2021? How much was it?
3. What state had the highest sales (in dollars) of men's products in 2021? How much was it?
4. What retailer purchased the most units in 2021? In 2020?

## Install and Import Necessary Libraries

In [1]:
# Install necessary packages
%pip install pandas numpy matplotlib seaborn scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Import ncessary packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import pprint
#import matplotlib.ticker as ticker

KeyboardInterrupt: 

In [ ]:
# Untruncate output
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Load Data and Create DataFrames

In [ ]:
# Load the product data from TABLE_PRODUCTS_885.csv into a Dataframe called products_df.
products_df = pd.read_csv('data/TABLE_PRODUCTS_885.csv', sep = '|')

# View the first few rows of products_df
products_df.head()

In [ ]:
# Load the retailer data from TABLE_RETAILER_885.csv into a Dataframe called retailers_df.
retailers_df = pd.read_csv('data/TABLE_RETAILER_885.csv')

# View the first few rows of retailers_df
retailers_df.head()

In [ ]:
# Load the sales data from TABLE_SALES_885.csv into a Dataframe called sales_df.
sales_df = pd.read_csv('data/TABLE_SALES_885.csv')

# View the first few rows of sales_df
sales_df.head()

In [ ]:
# Combine sales_df and products_df on the PRODUCT_ID column
sales_products_df = sales_df.merge(products_df, how = 'left', on = 'PRODUCT_ID')

# Combine the sales_products_df and retailers_df on the RETAILER_ID column
rush_sales_df = sales_products_df.merge(retailers_df, how = 'left', on = 'RETAILER_ID')

# View the first few rows of rush_sales_df
rush_sales_df.head()

## Inspect the Data

### Inspect the DataFrame Characteristics

In [ ]:
# Inspect the shape of rush_sales_df
rush_sales_df.shape

In [ ]:
# Inspect the info of rush_sales_df
rush_sales_df.info()

### Characteristics Check Results
+ The DataFrame has 16 columns with 10,271 rows.
+ No unwanted observations.
+ No unwanted features.
+ The ``INVOICE_DATE`` column is an object, we want to turn that into Date datatype.

In [ ]:
# Convert the INVOICE_DATE column to datetime format
rush_sales_df['INVOICE_DATE'] = pd.to_datetime(rush_sales_df['INVOICE_DATE'])

# View the information for the rush_sales_df
rush_sales_df.info()

### Identify any Unwanted Observations or Features

+ There are no Unwanted Observations or Features.

### Inspect for Missing Values

In [ ]:
# Traditional Null Values Check
rush_sales_df.isnull().sum()

There are traditional null values in the following columns:
+ PRICE_PER_PRODUCT: 2
+ RETAILER: 1
+ REGION: 1
+ STATE: 1
+ CITY: 1

In [ ]:
# Non-Traditional Categorical Null Values Check
# Identify by viewing unique values in each categorical column

# List of categorical columns to inspect by type
cat_columns = list(rush_sales_df.select_dtypes(include = 'object').columns)

# View unique values in each categorical column
for col in cat_columns:
    unique_values = rush_sales_df[col].dropna().astype(str).unique().tolist()
    unique_values.sort()
    print(f"Unique values in column '{col}':")
    pprint.pprint(unique_values)
    print("\n")

Some unique null values to address later on are as follows:
+ RETAILER_ID: 999999999
+ UNITS_SOLD: *** and 0

In [ ]:
# Non-Traditional Numerical Null Values Check
rush_sales_df.describe()

There is one column that has a non-traditional numerical null value.

+ PRICE_PER_UNIT has at least one value of 99999.

### Inspect for Duplicate Values

In [ ]:
# Inspect for Duplicate values
duplicate_rows = rush_sales_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

There are no duplicate rows in the dataframe.

### Inspect for Erroneous Values

There was one previously identified erroneous value in the dataframe.
+ SALES_METHOD: Ootlet instead of Outlet.

### Inspect for Outliers

In [ ]:
# Use the IQR method to identify outliers
# Write a function to calculate IQR and print rows with values that fall outside the IQR range
def detect_outliers_iqr(df, column):
    # Define Q1
    Q1 = df[column].quantile(0.25)
    # Define Q3
    Q3 = df[column].quantile(0.75)
    # Calculate IQR and define bounds
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Identify outliers
    outliers = (df[column] < lower_bound) | (df[column] > upper_bound)
    # Count the number of True values (Outliers)
    return outliers.sum()

In [ ]:
# Iterate over the numeric columns and identify outliers
for col in rush_sales_df.select_dtypes(include = 'number').columns:
    print(f"Number of outliers in column '{col}': {detect_outliers_iqr(rush_sales_df, col)}")
    print("\n")

There are 3 columns that have possible outliers:
+ YEAR
+ PRICE_PER_UNIT
+ OPERATING_MARGIN

In [ ]:
# Evaluate outliers in year.
year_outliers = rush_sales_df[(rush_sales_df['YEAR'] > 2021) | (rush_sales_df['YEAR'] < 2020)]

print(year_outliers)

There are no outliers in the year column, all years are 2020 or 2021.

In [ ]:
# Evaluate outliers in PRICE_PER_UNIT.
Q1 = rush_sales_df['PRICE_PER_UNIT'].quantile(0.25)
Q3 = rush_sales_df['PRICE_PER_UNIT'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = rush_sales_df[(rush_sales_df['PRICE_PER_UNIT'] < lower_bound) | 
                         (rush_sales_df['PRICE_PER_UNIT'] > upper_bound)]

outliers.sort_values(by = ['PRICE_PER_UNIT', 'REGION', 'RETAILER_ID', 'PRODUCT_ID'], ascending = False)

The PRICE_PER_PRODUCT outliers appear to not necessarily be outliers.  While they are the same product_name or category, they are different product id's and in different regions.  Some of the regions have a much higher cost of living in general.

In [ ]:
# Evaluate outliers in OPERATING_MARGIN.
Q1 = rush_sales_df['OPERATING_MARGIN'].quantile(0.25)
Q3 = rush_sales_df['OPERATING_MARGIN'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

margin_outliers = rush_sales_df[
    (rush_sales_df['OPERATING_MARGIN'] < lower_bound) | 
    (rush_sales_df['OPERATING_MARGIN'] > upper_bound)
]

margin_outliers.sort_values(by = ['OPERATING_MARGIN', 'REGION', 'RETAILER_ID', 'PRODUCT_ID'], ascending = False)

Not necessarily outliers that need to be fixed.  These plaices may all have operating margin's that are differnet since it appears that they are in different locations and regions.

## Clean the Data

### Address Unwanted Observations or Features

There are no Unwanted Observations or Features to address.

### Address any Erroneous Values

In [ ]:
# Replace the erroneous value for SALES_METHOD of Ootlet with Outlet
rush_sales_df.loc[rush_sales_df['SALES_METHOD'] == 'Ootlet', 'SALES_METHOD'] = 'Outlet'

### Address any Duplicate Values

There are no duplicate rows to address.

### Address any Missing Values

In [ ]:
# Check row with null values in RETAILER column
rush_sales_df[rush_sales_df['RETAILER'].isnull()]

Based on there being no desernable information to identify who the retailer is or where the retailer is located, and the fact this row is a singular row, I am going to exclude it from analysis.

In [ ]:
# Delete row from rush dataframe where RETAILER is null
rush_sales_df = rush_sales_df.drop(rush_sales_df[rush_sales_df['RETAILER'].isnull()].index)

In [ ]:
# Check row where PRICE_PER_UNIT is null
rush_sales_df[rush_sales_df['PRICE_PER_UNIT'].isnull()] 

In [ ]:
# Determine the mean PRICE_PER_UNIT for products that have a PRODUCT_ID of 20, OPERATING_MARGIN of 0.35, SALES_METHOD of In-store, REGION of Northeast, STATE of Vermont, and CITY of Burlington
rush_sales_df[rush_sales_df['PRODUCT_ID'] == 20].groupby(['OPERATING_MARGIN', 'SALES_METHOD', 'REGION', 'STATE', 'CITY'])['PRICE_PER_UNIT'].median()

# Filter the results by OPERATING_MARGIN of 0.35, SALES_METHOD of In-store, REGION of Northeast, STATE of Vermont, and CITY of Burlington
rush_sales_df[rush_sales_df['OPERATING_MARGIN'] == 0.35].groupby(['SALES_METHOD', 'REGION', 'STATE', 'CITY'])['PRICE_PER_UNIT'].median()

In [ ]:
# Perform the same process for products that have a PRODUCT_ID of 20, OPERATING_MARGIN of 0.50, SALES_METHOD of In-store, REGION of Northeast, STATE of Vermont, and CITY of Burlington
rush_sales_df[rush_sales_df['PRODUCT_ID'] == 20].groupby(['OPERATING_MARGIN', 'SALES_METHOD', 'REGION', 'STATE', 'CITY'])['PRICE_PER_UNIT'].median()
rush_sales_df[rush_sales_df['OPERATING_MARGIN'] == 0.50].groupby(['SALES_METHOD', 'REGION', 'STATE', 'CITY'])['PRICE_PER_UNIT'].median()

In [ ]:
# Replace the null values for the PRICE_PER_UNIT when OPERATING_MARGIN is 0.35 with 60.0.
rush_sales_df.loc[(rush_sales_df['PRICE_PER_UNIT'].isnull()) & (rush_sales_df['OPERATING_MARGIN'] == 0.35), 'PRICE_PER_UNIT'] = 60.0

# Replace the null values for the PRICE_PER_UNIT when OPERATING_MARGIN is 0.50 with 65.0.
rush_sales_df.loc[(rush_sales_df['PRICE_PER_UNIT'].isnull()) & (rush_sales_df['OPERATING_MARGIN'] == 0.50), 'PRICE_PER_UNIT'] = 65.0

In [ ]:
# Check out PRICE_PER_UNIT when value is 99999.
rush_sales_df[rush_sales_df['PRICE_PER_UNIT'] == 99999]

In [ ]:
# Perform the same process for products that have a PRODUCT_ID of 20, OPERATING_MARGIN of 0.4, SALES_METHOD of Online, RETAILER of Foot Locker, REGION of Northeast, STATE of New Hampshire, and CITY of Manchester
rush_sales_df[rush_sales_df['PRODUCT_ID'] == 20].groupby(['OPERATING_MARGIN', 'SALES_METHOD', 'REGION', 'STATE', 'CITY'])['PRICE_PER_UNIT'].median()
rush_sales_df[rush_sales_df['OPERATING_MARGIN'] == 0.40].groupby(['SALES_METHOD', 'REGION', 'STATE', 'CITY'])['PRICE_PER_UNIT'].median()

In [ ]:
# Replace the 99999 in PRICE_PER_UNIT with 45.0.
rush_sales_df.loc[rush_sales_df['PRICE_PER_UNIT'] == 99999, 'PRICE_PER_UNIT'] = 45.0

In [ ]:
# Check out rows where UNITS_SOLD are ***.
rush_sales_df[rush_sales_df['UNITS_SOLD'] == '***']

In [ ]:
# Determine like rows.
rush_sales_df[(rush_sales_df['OPERATING_MARGIN'] == 0.45) & (rush_sales_df['YEAR'] == 2021) & (rush_sales_df['SALES_METHOD'] == 'Online') & (rush_sales_df['REGION'] == 'South') & (rush_sales_df['RETAILER'] == 'Sports Direct') & (rush_sales_df['STATE'] == 'Alabama') & (rush_sales_df['PRODUCT_ID'] == 20)]

In [ ]:
# Replace the *** in UNITS_SOLD with 154 for row 1021.
rush_sales_df.loc[1021, 'UNITS_SOLD'] = 154

In [ ]:
# Check out rows where UNITS_SOLD are ***.
rush_sales_df[rush_sales_df['UNITS_SOLD'] == '***']

In [ ]:
# Determine like rows.
rush_sales_df[(rush_sales_df['OPERATING_MARGIN'] == 0.46) & (rush_sales_df['YEAR'] == 2021) & (rush_sales_df['SALES_METHOD'] == 'Outlet') & (rush_sales_df['REGION'] == 'Midwest') & (rush_sales_df['RETAILER'] == 'West Gear') & (rush_sales_df['STATE'] == 'Iowa') & (rush_sales_df['PRODUCT_ID'] == 20)]

Since this is another singular row, I am going to exclude it from the data.

In [ ]:
# Remove the record with UNITS_SOLD of ***.
rush_sales_df = rush_sales_df.drop(rush_sales_df[rush_sales_df['UNITS_SOLD'] == '***'].index)

In [ ]:
# Convert UNITS_SOLD TO INT.
rush_sales_df['UNITS_SOLD'] = rush_sales_df['UNITS_SOLD'].astype(int)

### Address any Outliers

The outliers identified above will be investigated further in the ``Determine Any Additional Trends or Insights`` section.

## Answer the Business Questions

In [ ]:
# Refresh list of columns with info.
rush_sales_df.info()

### 1. What product category (product) had the highest sales (in dollars) in 2021? How much did it sell?

In [ ]:
# Filter the results to only be results from the year 2021 and save to a df called rush_sales_2021.
rush_sales_2021 = rush_sales_df[rush_sales_df['YEAR'] == 2021].copy()

In [ ]:
# Add a column to rush_sales_2021 caled sales_total that is the product of PRICE_PER_UNIT and UNITS_SOLD.
rush_sales_2021.loc[:, 'SALES_TOTAL'] = rush_sales_2021['PRICE_PER_UNIT'] * rush_sales_2021['UNITS_SOLD']

In [ ]:
# Group by PRODUCT_NAME and sum the SALES_TOTAL column, and return the top 5 highest resullts.
rush_sales_2021.groupby('PRODUCT_NAME')['SALES_TOTAL'].sum().sort_values(ascending = False).head(5)

+ The product with the highest total sales (in dollars) in 2021 is ``Men's Street Footwear``.
+ The total amount sold was ``$23,288,610.00``.

In [ ]:
# Group and sum the sales by product
top_products = (
    rush_sales_2021
    .groupby('PRODUCT_NAME', as_index=False)['SALES_TOTAL']
    .sum()
    .sort_values(by='SALES_TOTAL', ascending=False)
    .head(3)
)

# Filter original dataset for only the top 3 products
filtered_data = rush_sales_2021[rush_sales_2021['PRODUCT_NAME'].isin(top_products['PRODUCT_NAME'])]

# Plot the bar chart
plt.figure(figsize=(10, 6))
sns.barplot(
    data=filtered_data,
    x='PRODUCT_NAME',
    y='SALES_TOTAL',
    estimator='sum',
    errorbar=None
)

# Format the plot
plt.title('Top 3 Products by Total Sales')
plt.xticks(rotation=45)
plt.ylabel('Total Sales (in $10M)')
plt.xlabel('Product Name')
plt.tight_layout()
plt.show()

### 2. What state had the highest sales (in dollars) of women's products in 2021? How much was it?

In [ ]:
# Create a GENDER column to identify if the product is for Men or Women (0 = Men, 1 = Women).
rush_sales_2021.loc[rush_sales_2021['PRODUCT_NAME'].str.contains("Men's", case=False, na=False), 'GENDER'] = 0
rush_sales_2021.loc[rush_sales_2021['PRODUCT_NAME'].str.contains("Women's", case=False, na=False), 'GENDER'] = 1

In [ ]:
# Filter for rows where GENDER == 1 (Women's).
womens_products = rush_sales_2021[rush_sales_2021['GENDER'] == 1]

# Group by STATE and sum the SALES_TOTAL column, and return the top results.
womens_products.groupby('STATE')['SALES_TOTAL'].sum().sort_values(ascending = False).head(5)

+ The state with the highest sales (in dollars) for women's products in 2021 was ``Maine``.
+ The amount of sales was ``$2,176,301.00``.

### 3. What state had the highest sales (in dollars) of men's products in 2021? How much was it?

In [ ]:
# Filter for rows where GENDER == 0 (Men's).
mens_products = rush_sales_2021[rush_sales_2021['GENDER'] == 0]

# Group by STATE and sum the SALES_TOTAL column, and return the results.
mens_products.groupby('STATE')['SALES_TOTAL'].sum().sort_values(ascending = False).head(5)

+ The state with the highest sales (in dollars) for men's products in 2021 was ``Delaware``.
+ The amount of sales was ``$2,334,300.00``.

### 4. What retailer purchased the most units in 2021? In 2020?

In [ ]:
# Filter the results to only be results from the year 2020 and save to a df called rush_sales_2020.
rush_sales_2020 = rush_sales_df[rush_sales_df['YEAR'] == 2020].copy()

# Add a column to rush_sales_2020 caled sales_total that is the product of PRICE_PER_UNIT and UNITS_SOLD.
rush_sales_2020.loc[:, 'SALES_TOTAL'] = rush_sales_2020['PRICE_PER_UNIT'] * rush_sales_2020['UNITS_SOLD']

# Create a GENDER column to identify if the product is for Men or Women (0 = Men, 1 = Women).
rush_sales_2020.loc[rush_sales_2020['PRODUCT_NAME'].str.contains("Men's", case=False, na=False), 'GENDER'] = 0
rush_sales_2020.loc[rush_sales_2020['PRODUCT_NAME'].str.contains("Women's", case=False, na=False), 'GENDER'] = 1

In [ ]:
# Group the rush_sales_2021 df by RETAILER and sum UNITS_SOLD, and then return the results.
rush_sales_2021.groupby('RETAILER')['UNITS_SOLD'].sum().sort_values(ascending = False).head(5)

In [ ]:
# Group the rush_sales_2020 df by RETAILER and sum UNITS_SOLD, and then return the Results.
rush_sales_2020.groupby('RETAILER')['UNITS_SOLD'].sum().sort_values(ascending = False).head(5)

+ The retailer who purchased the most units in 2021 was ``Foot Locker``.
+ The retailer who purchased the most units in 2020 was ``Amazon``.

## Determine Any Additional Trends or Insights

### What product had the lowest sales in 2021? How much sold?

In [ ]:
# Determine which product had the lowest sales in 2021.
rush_sales_2021.groupby('PRODUCT_NAME')['SALES_TOTAL'].sum().sort_values(ascending = True).head(5)

### What state had the lowest sales of women's products in 2021?

In [ ]:
# Deterimne which state had the lowest sales of women's products in 2021?
womens_products.groupby('STATE')['SALES_TOTAL'].sum().sort_values(ascending = True).head(5)

### What state had the lowest sales of men's products in 2021?

In [ ]:
# Determine which state had the lowest sales of men's products in 2021.
mens_products.groupby('STATE')['SALES_TOTAL'].sum().sort_values(ascending = True).head(5)

### What region had the highest sales in 2021? The lowest?

In [ ]:
# Determine which region had the highest sales in 2021.
rush_sales_2021.groupby('REGION')['SALES_TOTAL'].sum().sort_values(ascending = False).head(5)

In [ ]:
# Determine which region had the lowest sales in 2021.
rush_sales_2021.groupby('REGION')['SALES_TOTAL'].sum().sort_values(ascending = True).head(5)

### What retailer purchased the least units in 2021? 2020?

In [ ]:
# Determine which retailer purchased the least units in 2021.
rush_sales_2021.groupby('RETAILER')['UNITS_SOLD'].sum().sort_values(ascending = True).head(5)

In [ ]:
# Determine which retailer purchased the least units in 2020.
rush_sales_2020.groupby('RETAILER')['UNITS_SOLD'].sum().sort_values(ascending = True).head(5)

### What product had the least units sold in 2021? 2020?

In [ ]:
# Determine which product had the least units sold in 2021.
rush_sales_2021.groupby('PRODUCT_NAME')['UNITS_SOLD'].sum().sort_values(ascending = True).head(5)


In [ ]:
# Determine which product had the least units sold in 2020.
rush_sales_2020.groupby('PRODUCT_NAME')['UNITS_SOLD'].sum().sort_values(ascending = True).head(5)


### What retailer, including region, state, city had the highest operating marign? The lowest?

In [ ]:
# Determine the retailer with the highest operting margin.  Group by region, state, and city.
rush_sales_2021.groupby(['RETAILER', 'REGION', 'STATE', 'CITY'])['OPERATING_MARGIN'].sum().sort_values(ascending = False).head(5)

In [ ]:
# Determine the retailer with the lowest operting margin.  Group by region, state, and city.
rush_sales_2021.groupby(['RETAILER', 'REGION', 'STATE', 'CITY'])['OPERATING_MARGIN'].sum().sort_values(ascending = True).head(5)

### Using a bar chart show the relationship between region and product type.

In [ ]:
# Create a grouped bar chart using seaborn to show the total units sold by product and region.
plt.figure(figsize = (14, 7))
sns.barplot(
    data = rush_sales_df,
    x = 'PRODUCT_NAME',
    y = 'UNITS_SOLD',
    hue = 'REGION',
    estimator = 'sum',
    errorbar = None
)

# Format the plot
plt.title('Units Sold by Product and Region')
plt.xticks(rotation = 45)
plt.ylabel('Units Sold')
plt.xlabel('Product Name')

# Move legend to top, in a horizontal row
plt.legend(
    title = 'Region',
    loc = 'upper right',
    ncol = 5,
    frameon = False
)
plt.tight_layout()
plt.show()

### Using a bar chart show the relationship between product type and retailer.

In [ ]:
# Create a grouped bar chart using seaborn to show the total units sold by product and retailer.
plt.figure(figsize = (14, 7))
sns.barplot(
    data = rush_sales_df,
    x = 'PRODUCT_NAME',
    y = 'UNITS_SOLD',
    hue = 'RETAILER',
    estimator = 'sum',
    errorbar = None
)

# Format the plot
plt.title('Units Sold by Product and Retailer')
plt.xticks(rotation = 45)
plt.ylabel('Units Sold')
plt.xlabel('Product Name')

# Move legend to top, in a horizontal row
plt.legend(
    title = 'Retailer',
    loc = 'upper right',
    ncol = 5,
    frameon = False
)
plt.tight_layout()
plt.show()

### Create two line charts showing the sales for each retailer for 2021 and 2020.

In [ ]:
# Aggregate the total sales by retailer and month for 2021.
sales_trend_2021 = rush_sales_2021.groupby(['RETAILER', 'MONTH'])['UNITS_SOLD'].sum().sort_values(ascending = False).reset_index()

# Pviot the data to have months as index and retialers as columns
trend_pivot_2021 = sales_trend_2021.pivot(index = 'MONTH', columns = 'RETAILER', values = 'UNITS_SOLD')

# Sort by month
trend_pivot_2021.sort_index()

# Plot the trends
plt.figure()
for retailer in trend_pivot_2021.columns:
  plt.plot(trend_pivot_2021.index, trend_pivot_2021[retailer], label = retailer)

plt.xlabel('Month')
plt.ylabel('Units Sold')
plt.title('2021 Units Sold by Retailer and Month')
plt.legend()
plt.show()

In [ ]:
# Aggregate the total sales by retailer and month for 2020.
sales_trend_2020 = rush_sales_2020.groupby(['RETAILER', 'MONTH'])['UNITS_SOLD'].sum().sort_values(ascending = False).reset_index()

# Pviot the data to have months as index and retialers as columns
trend_pivot_2020 = sales_trend_2020.pivot(index = 'MONTH', columns = 'RETAILER', values = 'UNITS_SOLD')

# Sort by month
trend_pivot_2020.sort_index()

# Plot the trends
plt.figure()
for retailer in trend_pivot_2020.columns:
  plt.plot(trend_pivot_2020.index, trend_pivot_2020[retailer], label = retailer)

plt.xlabel('Month')
plt.ylabel('Units Sold')
plt.title('2020 Units Sold by Retailer and Month')
plt.legend()
plt.show()

### Create a box plot to show the relationship between price_per_unit and region for the produt types.

In [ ]:
# Create a grouped boxplot using seaborn to show the price per unit by region and product.
plt.figure(figsize = (10, 7))

sns.boxplot(
    data = rush_sales_df,
    x = 'REGION',
    y = 'PRICE_PER_UNIT',
    hue = 'PRODUCT_NAME'
)

plt.title('Price per Unit by Region and Product')
plt.xlabel('Region')
plt.ylabel('Price per Unit')
plt.xticks(rotation = 45)
plt.legend(title = 'Product Name', bbox_to_anchor = (1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### Create a bar graph showing the sales volume for each of the product types.

In [ ]:
# Add total_sales to rush_sales_df.
rush_sales_with_total = rush_sales_df.copy()

rush_sales_with_total['TOTAL_SALES'] = rush_sales_with_total['PRICE_PER_UNIT'] * rush_sales_with_total['UNITS_SOLD']

# Group by PRODUCT_NAME and YEAR, summing SALES_TOTAL
sales_summary = rush_sales_with_total.groupby(['PRODUCT_NAME', 'YEAR'])['TOTAL_SALES'].sum().reset_index()

# Pivot so each YEAR is a separate column
sales_pivot = sales_summary.pivot(index = 'PRODUCT_NAME', columns = 'YEAR', values = 'TOTAL_SALES').fillna(0)

# Plot the grouped bar chart
sales_pivot.plot(kind = 'bar', figsize = (14, 8))

# Formatting
plt.title('Total Sales by Product (2020 vs 2021)')
plt.xlabel('Product Name')
plt.ylabel('Total Sales in $10M')
plt.xticks(rotation = 45, ha = 'right')
plt.legend(title = 'Year')
plt.tight_layout()
plt.show()

### Create a bar graph showing the comparison between retailers for 2021 and 2020.

In [ ]:
# Group by RETAILER and YEAR, summing UNITS_SOLD
sales_summary = rush_sales_with_total.groupby(['RETAILER', 'YEAR'])['UNITS_SOLD'].sum().reset_index()

# Pivot so each YEAR is a separate column
sales_pivot = sales_summary.pivot(index = 'RETAILER', columns = 'YEAR', values = 'UNITS_SOLD').fillna(0)

# Plot the grouped bar chart
ax = sales_pivot.plot(kind = 'bar', figsize = (10, 6))

# Formatting
plt.title('Units Sold by Retailer (2020 vs 2021)')
plt.xlabel('Retailer')
plt.ylabel('Units Sold')
plt.xticks(rotation = 45, ha = 'right')
plt.legend(title = 'Year')

# Show full integer values on y-axis
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))

plt.tight_layout()
plt.show()